In [63]:
from urllib import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import string
from lxml import html

#Subject, Subject Code
file_list = pd.DataFrame(
    data=[['Economics', 'E'], 
          ['Geography', 'GEO'], 
          ['Government', 'GOV'], 
          ['Social Studies Skills', 'SSS'], 
          ['US History', 'US'], 
          ['World History', 'WH']],
          columns=['Subject', 'Subject Code'])
print file_list
subject = file_list.iloc[0,0]
subject_code = file_list.iloc[0,1]
standards_list = []
keys = ['Subject','Parent Text', 'Parent TID', 'Child Text', 'Child TID', 'Standard Code']

                 Subject Subject Code
0              Economics            E
1              Geography          GEO
2             Government          GOV
3  Social Studies Skills          SSS
4             US History           US
5          World History           WH


In [64]:
filename = r'file:///Users\Melissa\Documents\Udacity\Untitled Folder\High School ' + subject + '.htm'
file = urlopen(filename)

soup = BeautifulSoup(file, "lxml")
standards = soup.ul

In [65]:
#print standards

In [66]:
first_parent_text = standards.li.a.contents
print first_parent_text

[u'Economic Components']


In [67]:
first_parent = standards.li
print first_parent

<li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=3938544">Economic Components</a><ul><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=4128511">Agriculture</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703845">Economic Institutions</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703827">Economic Systems</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703828">Factors of Production</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703844">Financial Institutions</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703826">Free Market</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=4128512">Labor</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703843">Medium of Exchange</a></li><li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703841">Taxes</a></li><l

In [68]:
#second_parent = first_parent.next_sibling
#second_parent_text = first_parent.a.contents
#print second_parent_text
#print second_parent
parent = standards.li
parent_code = 1
children = parent.ul.findChildren(recursive=False)
#children = children.next_sibling
#print children[0].a.contents
print children[2].a.contents
#for child in children:
#    print child.contents
#children = children.next_sibling
print children

[u'Economic Systems']
[<li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=4128511">Agriculture</a></li>, <li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703845">Economic Institutions</a></li>, <li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703827">Economic Systems</a></li>, <li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703828">Factors of Production</a></li>, <li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703844">Financial Institutions</a></li>, <li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703826">Free Market</a></li>, <li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=4128512">Labor</a></li>, <li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703843">Medium of Exchange</a></li>, <li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_id=2703841">Taxes</a></li>, <li><a class="tree" href="/edit.epl?standard_id=35063&amp;topic_

In [70]:
#second_parent_children = second_parent.ul.findAll("a")
#print second_parent_children

parent = standards.li
parent_code = 1

while parent != None:
    parent_text = parent.a.contents
    parent_tid = parent.a['href'][-7:]
    #print parent
    children = parent.ul.findChildren(recursive=False)
    standard_code = subject_code + '.' + str(parent_code)
    child_code = 1
    standards_list.append(dict(zip(keys, [subject, parent_text, parent_tid, '', '', standard_code])))    
    #loop through children
    for child in children:
        child_text = child.a.contents
        child_tid = child.a['href'][-7:]
        standard_code = subject_code + '.' + str(parent_code) + '.' + str(child_code)
        standards_list.append(dict(zip(keys, [subject, parent_text, parent_tid, child_text, child_tid, standard_code])))    
        child_code += 1
    parent = parent.next_sibling
    parent_code += 1
print standards_list

[{'Parent Text': [u'Economic Components'], 'Child TID': '', 'Parent TID': '3938544', 'Standard Code': 'E.1', 'Child Text': '', 'Subject': 'Economics'}, {'Parent Text': [u'Economic Components'], 'Child TID': '4128511', 'Parent TID': '3938544', 'Standard Code': 'E.1.1', 'Child Text': [u'Agriculture'], 'Subject': 'Economics'}, {'Parent Text': [u'Economic Components'], 'Child TID': '2703845', 'Parent TID': '3938544', 'Standard Code': 'E.1.2', 'Child Text': [u'Economic Institutions'], 'Subject': 'Economics'}, {'Parent Text': [u'Economic Components'], 'Child TID': '2703827', 'Parent TID': '3938544', 'Standard Code': 'E.1.3', 'Child Text': [u'Economic Systems'], 'Subject': 'Economics'}, {'Parent Text': [u'Economic Components'], 'Child TID': '2703828', 'Parent TID': '3938544', 'Standard Code': 'E.1.4', 'Child Text': [u'Factors of Production'], 'Subject': 'Economics'}, {'Parent Text': [u'Economic Components'], 'Child TID': '2703844', 'Parent TID': '3938544', 'Standard Code': 'E.1.5', 'Child Tex

In [71]:
standards_list_df = pd.DataFrame(standards_list)
print standards_list_df

   Child TID                               Child Text Parent TID  \
0                                                        3938544   
1    4128511                            [Agriculture]    3938544   
2    2703845                  [Economic Institutions]    3938544   
3    2703827                       [Economic Systems]    3938544   
4    2703828                  [Factors of Production]    3938544   
5    2703844                 [Financial Institutions]    3938544   
6    2703826                            [Free Market]    3938544   
7    4128512                                  [Labor]    3938544   
8    2703843                     [Medium of Exchange]    3938544   
9    2703841                                  [Taxes]    3938544   
10   2703835                             [Technology]    3938544   
11                                                       3938542   
12   2703846                         [Business Cycle]    3938542   
13   2703830                                 [Ch

In [72]:
#output to Excel
output = subject + "Export.xlsx"

writer = pd.ExcelWriter(output)
standards_list_df.to_excel(writer, sheet_name='Sheet1')
writer.save()